# Visualize Landprices


In [85]:
import pandas as pd
import geopandas as gpd
import os
import osmnx as ox 

### Vector data 
We start by reading and quickly visualising vector data

### Read raster data

We read the census data that is classified in a grid, then we read the grid as a geopandas file.

In [89]:
ZENSUS_PATH = r"../data/raw/2 Zensus/"
NEIGHBORHOOD_PATH =  r"../data/raw/3 Neighborhoods"
LAND_PRICES_PATH = r"../data/raw/1 Land Prices"
CITY_NAMES=["Berlin","Bremen","Dresden","Frankfurt","Köln"]
zensus_files= os.listdir(ZENSUS_PATH)
neighborhood_files= os.listdir(NEIGHBORHOOD_PATH)
landprices_files= os.listdir(LAND_PRICES_PATH)

def get_file_names(): 
    city_files={}
    for city_name in CITY_NAMES:
        
        csv_fpaths= [os.path.join(ZENSUS_PATH,fpath) for fpath in zensus_files if ( city_name in fpath ) and (fpath.endswith(".csv")) ]
        gpkg_fpaths= [os.path.join(ZENSUS_PATH,fpath) for fpath in zensus_files if ( city_name in fpath ) and (fpath.endswith(".gpkg")) ]
        
        neighbourhood_fpaths= [os.path.join(NEIGHBORHOOD_PATH,fpath) for fpath in neighborhood_files if ( city_name in fpath ) and (fpath.endswith(".gpkg")) ]
        landprices_fpaths= [os.path.join(LAND_PRICES_PATH,fpath) for fpath in landprices_files if ( city_name in fpath ) ]

        city_files[city_name]= (csv_fpaths,gpkg_fpaths,neighbourhood_fpaths,landprices_fpaths)
    return city_files

def combine_data_within_city(city_files):
    city_merged_data={}
    for city_name, (csv_fpaths,gpkg_fpaths,neighbourhood_fpaths,landprices_fpaths) in city_files.items():
        df_list_city=[]
        for csv_fpath in csv_fpaths:
            df = pd.read_csv(csv_fpath, sep=";", encoding="utf-8-sig").drop(columns="Unnamed: 0")
            df_list_city.append(df)

        from functools import reduce
        zensus_csv_merged = reduce(lambda df1,df2: pd.merge(df1,df2,on="Grid_Code"), df_list_city)
        
        # read grid
        grid_city = gpd.read_file(gpkg_fpaths[0])
        grid_city = grid_city.merge(zensus_csv_merged, on = "Grid_Code")

        # merge files
        prices_city = pd.read_csv(landprices_fpaths[0], sep=";", encoding= "utf-8-sig").drop(columns="Unnamed: 0")
        neighborhood_city = gpd.read_file(neighbourhood_fpaths[0])
        neighborhood_city = neighborhood_city.merge(prices_city, on = "Neighborhood_FID", how = "inner")

        neighborhood_city_4326 = neighborhood_city.to_crs(epsg = 4326)
        amenity_features = get_amenity_features(neighborhood_city_4326)

        # Perform spatial join using sjoin
        merged_data = gpd.sjoin(grid_city, neighborhood_city, how='left', op='intersects')
        merged_data = merged_data.drop(columns=["City_Name_y", "City_Code_right"]).rename(columns={"City_Name_x":"City_Name", "City_Code_left":"City_Code"})
        city_merged_data[city_name] = merged_data
    return city_merged_data

def concat_city_dataframes(city_merged_data):
    col_list=[]
    for city_name,df in city_merged_data.items():
        print(df.shape)
        col_list.append(df.columns.to_list())
    common_cols=list(set(col_list[0]).intersection(*col_list))

    grid_level_df_list = [df[common_cols] for df in city_merged_data.values()]
    grid_level_all_cities = pd.concat(grid_level_df_list, axis=0)
    return grid_level_all_cities

def get_amenity_features(neighborhoods):
    # Define the tag to extract (amenity=restaurant)
    tag = {'amenity':True}

    # Define an empty list to store the results
    results = {}

    # Loop over each neighborhood and extract the restaurants
    for i, nb_name in enumerate(neighborhoods.Neighborhood_Name):
        nb = neighborhoods.loc[neighborhoods.Neighborhood_Name == nb_name]
        restaurants = ox.geometries.geometries_from_polygon(polygon=nb.geometry.iloc[0], tags=tag)
        # print(f'Processed {i+1}/{len(neighborhoods)} neighborhoods ({nb_name}): found {restaurants.shape[0]} restaurants')
        results[nb_name] = restaurants

    nb_results = []

    for i, nb_name in enumerate(neighborhoods.Neighborhood_Name):
        nb_result = pd.DataFrame(results[nb_name].amenity.value_counts().to_dict(), index=[i])
        nb_result["Neighborhood_Name"]=nb_name
        #nb_result["City_Name"]=neighborhoods.City_Name
        nb_results.append(nb_result)

    # Combine the results into a single GeoDataFrame
    combined_results = pd.concat(nb_results, ignore_index=True)
    

    DROPNA_TRESH=0.8
    combined_results = combined_results.loc[:, combined_results.isnull().mean() < DROPNA_TRESH]

    return combined_results 

In [90]:
city_files= get_file_names()


city_merged_data = combine_data_within_city(city_files)

grid_level_all_cities= concat_city_dataframes(city_merged_data)


c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and

(96838, 194)
(35637, 196)
(37357, 194)
(27329, 193)
(45330, 194)


In [94]:
Neighborhood_FID_lookup = grid_level_all_cities.filter(regex="Neighborhood|City").drop_duplicates()

In [97]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 50FA-7BE6

 Directory of c:\Users\BerkÖztürk\ifohack-landers-group\notebooks

30/04/2023  11:01    <DIR>          .
30/04/2023  10:59    <DIR>          ..
28/04/2023  20:27                 0 .gitkeep
29/04/2023  21:51            33.150 baseline.ipynb
30/04/2023  09:10             5.583 bokeh_vis.ipynb
30/04/2023  09:59    <DIR>          cache
30/04/2023  09:10            29.117 connectedness_neighbourhoods.ipynb
29/04/2023  12:13             7.236 data analysis.ipynb
28/04/2023  21:31    <DIR>          data_example
30/04/2023  09:46            10.080 merge_all_city_data.ipynb
30/04/2023  09:10        16.171.008 merged_data.gpkg
29/04/2023  14:52    <DIR>          mlruns
29/04/2023  18:03            50.910 oms_features.ipynb
30/04/2023  09:10    <DIR>          quickstart_notebooks
30/04/2023  09:10            18.360 segregation.ipynb
30/04/2023  09:10             4.278 streamlit.py
              10 File(s)     16.329.722 bytes
  

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2341528fd00>
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


In [98]:
Neighborhood_FID_lookup.to_csv("../data/interim/Neighborhood_FID_lookup.csv",index=False)

In [ ]:
splitted_area_names=pd.Series(grid_level_all_cities.Area_Types.unique()).str.split("_")
unique_area_cols = pd.Series(np.concatenate(splitted_area_names)).unique().tolist()
for unique_area_col in unique_area_cols:
    grid_level_all_cities['is_{}'.format(unique_area_col)]= grid_level_all_cities.Area_Types.str.contains(unique_area_col).astype(int)
grid_level_all_cities = grid_level_all_cities.drop(columns='Area_Types')

In [51]:
import numpy as np


numeric_cols = grid_level_all_cities.select_dtypes(include=['int32','int64']).columns
non_numeric_cols = grid_level_all_cities.select_dtypes(exclude=['int32','int64']).columns

agg_operations= dict(zip(numeric_cols, ['mean']*len(numeric_cols) ))
agg_operations["Land_Value"]="first"
grid_level_all_cities_neighborhood_level =  grid_level_all_cities.groupby(['City_Name','Neighborhood_Name']).agg(agg_operations).reset_index()
#grid_level_all_cities_neighborhood_level= grid_level_all_cities_neighborhood_level.drop(columns='index_right')#.describe()

In [53]:
grid_level_all_cities_neighborhood_level.to_csv('../data/interim/nb_level_merged_all_cities.csv',index=False)

In [ ]:
# Save merged data to file
grid_level_all_cities.to_file('../data/interim/grid_level_merged_all_cities.gpkg', driver='GPKG')

In [70]:
grid_level_all_cities_neighborhood_level.shape

(421, 186)

In [65]:
amenity_data={}
for city_name, (csv_fpaths,gpkg_fpaths,neighbourhood_fpaths,landprices_fpaths) in city_files.items():
    neighborhood_city = gpd.read_file(neighbourhood_fpaths[0])
    neighborhood_city_4326 = neighborhood_city.to_crs(epsg = 4326)
    amenity_features = get_amenity_features(neighborhood_city_4326)
    amenity_data[city_name] = amenity_features


In [66]:
amenity_data_all_cities = pd.concat(amenity_data.values(), axis=0)


In [72]:
amenity_data_all_cities.filter(regex="Neig|City|AGS")

,Neighborhood_Name,City_Name
0,Mitte,Berlin
1,Moabit,Berlin
2,Hansaviertel,Berlin
3,Tiergarten,Berlin
4,Wedding,Berlin
...,...,...
81,Fühlingen,Köln
82,Blumenberg,Köln
83,Libur,Köln
84,Poll,Köln


In [81]:
a = pd.merge(grid_level_all_cities_neighborhood_level,amenity_data_all_cities,how="inner",on=["Neighborhood_Name","City_Name"])

In [83]:
a.to_csv('../data/interim/nb_level_merged_all_cities_with_amenties.csv',index=False)